## Einteilung von Prüfungen der Grundvorlesungen

Dieses Notebook erstellt auf Grundlage der Anmeldungen in HisInOne, den Prüferwünschen von Studierenden und den von Prüfern angegebenen Zeiten einen Prüfungsplan. Nötige Input-Dateien heißen hier A_..., B_..., die Output-Dateien sind mit 01_..., 02_..., nummeriert.

## 1. Einlesen der Anmeldungen aus HisInOne

Wir lesen zunächst die Daten aus HisInOne aus einer xls-Tabelle hisinone_xls ein. Diese hat die folgende Form: Das xls-File enthält ab Zeile 2:

- In Spalte A eine Prüfungsnummer der Form 07LE23PL-<prüfungsordnung>-P-<prüfungsfach>, wobei <prüfungsordnung> etwa 2HfB21 ist, und <prüfungsfach> etwa Ana oder LA ist. Aus dieser Spalte wird mit Hilfe der Datei _prüfungsnummern_csv_ die Information generiert um welchen Studiengang und welche Prüfung es sich handelt.
- In Spalte B der Nachname des Prüflings.
- In Spalte C den Vornamen des Prüflings.
- In Spalte D die Immatrikulationsnummer.

Falls mehrere xls-Dateien aus HisInOne vorliegen, kopiert man am besten die entsprechenden Spalten
in eine Datei zusammen. Alle diese Anmeldungen werden in einer Datei _anmeldungen_hisinone_csv_ abgelegt.

In [ ]:
# Input
anmeldungen_from_hisinone_csv = 'A_HisInOne.csv'
prüfungsnummern_csv = "B_pruefungsnummern.csv"

# Output
anmeldungen_hisinone_csv = '01_HisInOne.csv'

# Ausführung
# Here is a python script where all important functions are stored:
%run einteilung.py

anmeldungen_hisinone = get_anmeldungen_from_hisinone(anmeldungen_from_hisinone_csv, prüfungsnummern_csv)

#Store the csv File
anmeldungen_hisinone.to_csv(anmeldungen_hisinone_csv, encoding='utf-8', index=False)
print(f"Das Ergebnis liegt in {anmeldungen_hisinone_csv} vor.")


## 2. Einlesen der Anmeldungen aus dem www-Interface

Zunächst werden aus dem Ordner _anmeldung_www_folder_ alle Dateien geladen. Jeder Prüfling hat dort ein File, und dieses File hat nur eine Zeile von der Form

{'wunsch1': 'junker', 'matrnr': '4999984', 'wunsch3': 'martin', 'vorname': 'Viktor', 'studiengang': 'bsc', 'nachname': 'Krause', 'pruefung': 'analysis', 'wunsch2': 'criens'}

Ist pruefung==analysis, so wird unterschieden zwischen studiengang=lehramt (dann Prüfung Analysis I+II) und studiengang=bsc (dann Prüfung in Analysis I-III). 

Das Ergebnis wird in zwei Dateien _anmeldungen_www_csv_ und _anmeldungen_www_ohne_duplikate_csv_ ausgegeben.

In [ ]:
# Input 
# Alle Anmeldungen liegen im Arbeitsverzeichnis im Order anmeldungen_www_folder. 

anmeldungen_www_folder = "anmeldungen_www"

# Output
anmeldungen_www_csv = "02_anmeldungen_www.csv" 
anmeldungen_www_ohne_duplikate_csv = "03_anmeldungen_www_ohne_duplikate.csv" 

# Ausführung
# Here is a python script where all important functions are stored:
%run einteilung.py

anmeldungen_www = get_anmeldungen_from_www("anmeldungen_www")

# Store the csv File
anmeldungen_www.to_csv(anmeldungen_www_csv, encoding='utf-8', index=False)
print(f"Das Ergebnis liegt in {anmeldungen_www_csv} vor.")

# Now delete duplicates, i.e. for every pair (Matrikelnummer, Fach) choose the largest id
anmeldungen_www_ohne_duplikate = anmeldungen_www.drop_duplicates(subset = ['Matrikelnummer', 'Fach'], keep = 'last')
anmeldungen_www_ohne_duplikate.reset_index(drop=True, inplace=True)

print(f"{len(anmeldungen_www_ohne_duplikate)} übrig nach Entfernen von Duplikaten.")
# Store the csv File
anmeldungen_www_ohne_duplikate.to_csv(anmeldungen_www_ohne_duplikate_csv, encoding='utf-8', index=False)
print(f"Das Ergebnis liegt in {anmeldungen_www_ohne_duplikate_csv} vor.")


## 3. Abgleich der HisInOne-Anmeldungen mit den www-Anmeldungen

Es werden nun zwei Listen ausgegeben:

1. Anmeldungen von Studierenden, die sich über HisInOne angemeldet haben, aber keine Prüferwünsche angegeben haben.
2. Abgabe von Prüferwünschen von Studierenden, die sich nicht auf HisInOne angemeldet haben.

Es kann sińnvoll sein, den Input-Dateien nun die Endung _final.csv_ zu geben, falls man Dateien, die oben erzeugt wurden, per Hand editiert hat. Dies kann verhindern, dass man aus Versehen bereits verrichtete Arbeit überschreibt.

In [ ]:
# Input-Dateien von oben
anmeldungen_hisinone_csv = '01_HisInOne.csv'
anmeldungen_www_ohne_duplikate_csv = "03_anmeldungen_www_ohne_duplikate.csv" 

# Output: Siehe unter diesem Chunk

# Ausführung
# Here is a python script where all important functions are stored:
%run einteilung.py

compare_anmeldungen_hisinone_www(anmeldungen_hisinone_csv, anmeldungen_www_ohne_duplikate_csv)


## 4. Vergabe der Prüfungen

In der Datei _prüferwünsche_csv_ werden die Daten der Prüfer gesammelt. Jede Zeile ist von der Form

Kurzname, Vorname, Nachname, Prüfungszahl, Analysis I+II, Ana1ysis I-III, LA

also etwa

bartels, Sören, Bartels, 12, 1, 1, 0

Dabei bedeutet der vierte Eintrag die maximale Anzahl an Prüfungen. Einträge 5-7 geben an, ob
der Prüfer Analysis|2HfB (d.h. Analysis 1, 2), Analysis|BSc (d.h. Analysis 1, 2, 3) und/oder 
Lineare Algebra prüft.
Hinweis: Dieses Format weicht vom Format des bisherigen Programms ab!

Es erfolgt die Optimierung mit Hilfe des Vektors _weights_, etwa

weights = (0, 2, 5, 10, 1000)

Dies bedeutet, dass ein Prüfling, der zu _wunsch1_ zugeordnet wird, die Zielfunktion nicht, der zu _wunsch2_ zugeordnet wird um 2, der zu _wunsch3_ zugeordnet wird um 5, der zu keinem der Prüferwünsche zugeordnet wird um 10 (bzw. 1000) erhöht, falls der Prüfer dieses Fach (nicht) anbietet. Diese Gewichte sind die Basis der folgenden Optimierung der Zielfunktion.

Es werden dabei alle Prüflinge in _03_anmeldungen_hisinone_csv_ eingeteilt, wobei die Prüferwunsch-Daten aus _anmeldungen_www_ohne_duplikate_csv_ eingelesen werden. Die Ausgabe einiger Statistiken, etwa wieviele Prüflinge zum Erst-, 
Zweit-, und Drittwunsch zugeordnet wurden. Weiter wird _04_zuordnungen_csv_ erzeugt, die _03_anmeldungen_hisinone_csv_ um die zusätzlichen Prüflinge aus _anmeldungen_www_ohne_duplikate_csv_ und eine Spalte mit dem Kurznamen des Prüfers erweitert. Außerdem wird nach Prüfern und Fächern sortiert.

Noch etwas genauer geht der Algorithmus wiefolgt vor:
- Ordne die Prüflingen den Prüfern anhand weights zu;
- Reduziere die Anzahl der Prüfungen pro Prüfer (angefangen mit dem Prüfer mit der momentan maximalen Anzahl an Prüfungen;
- Stoppe, wenn entweder _max_iterations_ erreicht wurde, oder die Zielgröße um mehr als _max_malus_ gestiegen ist, oder weniger
Prüfungsslots and Prüfungen verfügbar sind.


In [ ]:
# Input
prüferwünsche_csv = "C_prueferwuensche.csv"
anmeldungen_hisinone_csv = '01_HisInOne.csv'
anmeldungen_www_ohne_duplikate_csv = "03_anmeldungen_www_ohne_duplikate.csv" 

# Konfiguration des Algorithmus
# weights gibt die Gewichte an, falls zu wunsch1-3 oder zu keinem der drei Wünsche zugeordnet wird, oder zu einem andere Prüfer, 
# oder zu einem Prüfer, der das Fach gar nicht prüft.
weights = np.array([0, 2, 5, 10, 1000])
# Maximale Anzahl an Iterationen, bei denen die maximale Anzahl pro Prüfer reduziert wird um die Prüfungne möglichst gleichmäßig zu verteilen.
max_iterations = 200
# Maximaler Malus für gleichverteilte Prüfungen
max_malus = 0

# Output 
zuordnungen_csv = "04_zuordnungen.csv"

# Ausführung
# Wie oben müssen wir das wichtige Skript laufen lassn.
%run einteilung.py

zuordnungen, gesamtkosten = get_zuordnungen(prüferwünsche_csv, anmeldungen_hisinone_csv, anmeldungen_www_ohne_duplikate_csv, weights, max_iterations=max_iterations, max_malus = max_malus)
print(f"Gesamtkosten: {gesamtkosten}")
zuordnungen.to_csv(zuordnungen_csv, encoding='utf-8', index=False)
print(f"Alle Prüflinge erfolgreich zugeordnet. Das Ergebnis ist in {zuordnungen_csv}.")
get_statistiken_from_zuordnungen(zuordnungen_csv, prüferwünsche_csv)

## 5. Einteilung der Prüfungsslots

Nun ist _prüfungsslots_csv_ eine Datei mit dem Format 

Kurzname, Datum, Uhrzeit, Raum

bei der Prüfer angeben, welche Prüfungsslots sie anbieten. (Das Datum muss in der Form (DD.MM.YY), die Uhrzeit in der Form HH:MM:SS angegeben sein.) 

Es weden nun Prüfungsanzahlen pro Prüfer aus _prüferwünsche_csv_, _zuordnungen_csv_ und _prüfungsslots_csv_ verglichen. Es sollte für jeden Prüfer _prüferwünsche_csv_ >= _#prüfungsslots_csv_>= #_zuordnungen_csv_ gelten. Ist die erste Ungleichung verletzt, wird eine Warnung ausgegeben. Ist die zweite Ungleichung verletzt, wird ein Fehler ausgegeben und es findet keine Einteilung statt.

In [ ]:
# Input
zuordnungen_csv = "04_zuordnungen.csv"
prüferwünsche_csv = "C_prueferwuensche.csv"
prüfungsslots_csv = "D_pruefungsslots.csv"

# Output
prüfungsslots_mit_prüfungs_ids_csv = "05_pruefungsslots_mit_pruefungs_ids.csv"

# Ausführung
# Wie oben müssen wir das wichtige Skript laufen lassn.
%run einteilung.py
    
err = check_prüferwünsche_und_slots(prüferwünsche_csv, zuordnungen_csv, prüfungsslots_csv)    

if(err == True):
    print(f"Eine Zuordnung der Prüfer auf die Prüfungsslots ist nicht möglich. ")
else:
    prüfungsslots_mit_prüfungs_ids = make_zuordnungen_zu_slots(prüfungsslots_csv, zuordnungen_csv)
    prüfungsslots_mit_prüfungs_ids.to_csv(prüfungsslots_mit_prüfungs_ids_csv, encoding='utf-8', index=False)
    print(f"Alle Prüfungsslots erfolgreich zugeordnet. Das Ergebnis ist in {prüfungsslots_mit_prüfungs_ids_csv}.")


## 6. Erstellung des Prüfungsplans

Basierend auf dem Output des letzten Chunks _prüfungsslots_mit_prüfungs_ids_csv_ wird nun der Prüfungsplan erstellt. Dieser Output hatte die Form

Kurzname, Datum, Uhrzeit, Raum, prüfungs_id

Es muss also noch der Kurzname des Prüfers in den vollen Namen ergänzt werden, und weiter Details der prüfungs_id um die Daten des Prüflings und der Prüfung ergänzt werden. Ergebnis ist einerseits eine Excel-Tabelle mit dem fertigen Prüfungsplan, aber auch eine csv-Datei mit allen Informationen.


In [ ]:
# Input
prüfungsslots_mit_prüfungs_ids_csv = "05_pruefungsslots_mit_pruefungs_ids.csv"
zuordnungen_csv = "04_zuordnungen.csv"
prüferwünsche_csv = "C_prueferwuensche.csv" # Hier stehen Vor- und Nachname der Prüfer

# Output
prüfungsplan_csv = "06_pruefungsplan.csv"
prüfungsplan_xls = "06_pruefungsplan.xlsx"

# Ausführung
# Wie oben müssen wir das wichtige Skript laufen lassn.
%run einteilung.py

prüfungsplan = make_prüfungsplan(prüfungsslots_mit_prüfungs_ids_csv, zuordnungen_csv, prüferwünsche_csv)
prüfungsplan.to_csv(prüfungsplan_csv, encoding='utf-8', index=False)

writer = pd.ExcelWriter(prüfungsplan_xls)
prüfungsplan.to_excel(writer, sheet_name="Prüfungsplan", index=False)
worksheet = writer.sheets["Prüfungsplan"]  # pull worksheet object

for idx, col in enumerate(prüfungsplan):  # loop through all columns
    series = prüfungsplan[col]
    max_len = max((
    series.astype(str).map(len).max(),  # len of largest item
        len(str(series.name))  # len of column name/header
        )) + 1  # adding a little extra space
    worksheet.set_column(idx, idx, max_len)  # set column width
writer.close()

#prüfungsplan.to_excel(prüfungsplan_xls, index=False)
print(f"Erstellen des Prüfungsplans erfolgreich. Output in in {prüfungsplan_csv} und {prüfungsplan_xls}.")

## 7. Erstellung der Niederschriften

Nun werden die Niederschriften in _niederschriften_ordner_ erstellt. Dies geschieht mit _pdflatex_. Alle Dateien außer die pdf-Dateien werden sofort wieder gelöscht. Es bleiben nur die Niederschriften im Ordner, _niederschriften_alle_pdf_ im aktuellen Ordner und _latex_output_, wo der gesamte pdflatex-Output ausgegeben wird. 

Hierzu wird das Template _niederschriften_template_tex_ verwendet. (Hier wird ein Hintergrundbild eingebunden, das so aussieht wie eine leere Niederschrift.)

In [ ]:
# Input
prüfungsplan_csv = "06_pruefungsplan.csv"
niederschriften_ordner = "niederschriften"
niederschriften_template_tex = "niederschrift-template.tex"
latex_output = "07_output.log"

# Output
niederschiften_alle_pdf = "07_alle_niederschriften.pdf"

# Ausführung
# Wie oben müssen wir das wichtige Skript laufen lassn.
%run einteilung.py
    
make_niederschriften(prüfungsplan_csv, niederschriften_ordner, niederschriften_template_tex, latex_output, niederschiften_alle_pdf)
